<a href="https://colab.research.google.com/github/onurdurhan/Mytest/blob/master/ground_state.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import matplotlib.pyplot as plt 
import numpy as np                                  
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import torch
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn as nn
import math
import torch.nn.functional as F

In [32]:
if torch.cuda.is_available():
  print("Cuda (GPU support) is available and enabled!")
  device = torch.device("cuda")
else:
  print("Cuda (GPU support) is not available :(")
  device = torch.device("cpu")

Cuda (GPU support) is not available :(


In [33]:
def riemann(b_n,psy,x):
  dx=abs(x[-1])/len(x)
  projection=sum(b_n*psy*dx)
  return projection

In [34]:
#x=np.linspace(0., 1., 100)
x = torch.unsqueeze(torch.linspace(0., 1., 100), dim=1)
x=Variable(x)
#x = Variable(torch.Tensor(np.array(x)))

In [35]:
class TwoLayerNet(nn.Module):
    def __init__(self, D, H, C):
        super(TwoLayerNet, self).__init__()
        #torch.manual_seed(501)        
        self.fc1 = nn.Linear(D,H)
        self.fc = nn.Linear(H,H)
        self.fc2 = nn.Linear(H,C)

    def forward(self, x):
      print("before",x.shape)
      x = self.fc1(x)
      x = F.relu(x)
      x = self.fc(x)
      x = F.relu(x)
      x = self.fc2(x)
      return x

In [36]:
def hamiltonian_loss(output,N,x):
  loss=0.0
  a=1.
  amplitude=0.0
  for n in range(1,N):
    H_nn=(math.pi**2)*(n**2)/2.
    decomposition=riemann(math.sqrt(2./a)*torch.sin(n*x*torch.tensor(math.pi)/a),output,x)
    loss+=H_nn*abs(decomposition)**2
    amplitude+=decomposition**2
  loss=loss/(amplitude)
  return loss 

In [37]:
def perturbed_loss(output,N,x):
  a=1.
  M=100
  N=100
  alpha = 8.
  decompositions=torch.zeros(M)
  x_nm=torch.zeros((N,M))
  for n in range(N):
    for m in range(M):
      decompositions[m]= riemann(math.sqrt(2./a)*torch.sin((m+1)*x*math.pi/a),output,x)
      if not m == n:
        x_nm[n][m] = (math.cos((n-m)*math.pi)-1)/(n-m)**2-(math.cos((n+m)*math.pi)-1)/(n+m)**2
        x_nm[n][m] = x_nm[n][m]*a/math.pi**2
      if m == n :
        x_nm[n][m] = a/2.
  decompositions=torch.tensor(decompositions)
  x_loc = x_nm*decompositions
  x_loc=torch.tensor(x_loc)
  loss = alpha*torch.sum((x_loc.sum(dim=1)*decompositions**2))/torch.sum(decompositions**3)
  loss = Variable(loss, requires_grad = True)+hamiltonian_loss(output,N,x)
  return loss

In [38]:
def train(model, criterion, optimizer, epochs,x, verbose=True):
  loss_history=[]
  # Train the model
  for epoch in range(epochs):

    outputs = model(x)

    optimizer.zero_grad()
  
    loss = criterion(outputs, 100,x)
    
    loss.backward()
    
    optimizer.step()
    loss_history.append(loss.item())
    if verbose: print(f'Epoch {epoch} / {epochs}: avg. loss of last 5 iterations {np.sum(loss_history[:-6:-1])/5}')
  return loss_history   

In [11]:
import torch.optim as optim

D = 1
C = 1 
H = 1000 s
epochs=1000
learning_rate=0.0002
model = TwoLayerNet(D, H, C)

#criterion = hamiltonian_loss
#criterion = perturbed_loss
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
loss_history=train(model,criterion,optimizer,epochs,x)
plt.plot(loss_history)
plt.show()

NameError: ignored

In [ ]:
model.eval()
predicted = model(x)
predicted = predicted.data.numpy()
plt.plot(x,predicted)
plt.show()

In [39]:
class PerturbedNet(nn.Module):
    def __init__(self, D, H1, H2):
        super(PerturbedNet, self).__init__()
        #torch.manual_seed(501)        
        self.fc1 = nn.Linear(D,H1)
        self.fc = nn.Linear(H1,H2)
        self.fc2 = nn.Linear(H2,1)

    def forward(self, x):
      x = self.fc1(x)
      x = F.relu(x)
      x = self.fc(x)
      x = F.relu(x)
      x = self.fc2(x)
      return x

In [ ]:
D = 1
H1 = 500
H2 = 100
epochs=100
learning_rate=0.0000002
model = PerturbedNet(D, H1, H2)
criterion = perturbed_loss
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
loss_history=train(model,criterion,optimizer,epochs,x)
plt.plot(loss_history)
plt.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch 0 / 100: avg. loss of last 5 iterations 23.56107940673828
Epoch 1 / 100: avg. loss of last 5 iterations 46.99229888916015
Epoch 2 / 100: avg. loss of last 5 iterations 70.29753875732422
Epoch 3 / 100: avg. loss of last 5 iterations 93.48065948486328
Epoch 4 / 100: avg. loss of last 5 iterations 116.5421356201172
Epoch 5 / 100: avg. loss of last 5 iterations 115.92152099609375
Epoch 6 / 100: avg. loss of last 5 iterations 115.30795135498047
Epoch 7 / 100: avg. loss of last 5 iterations 114.69684600830078
Epoch 8 / 100: avg. loss of last 5 iterations 114.08281707763672
Epoch 9 / 100: avg. loss of last 5 iterations 113.48115997314453
Epoch 10 / 100: avg. loss of last 5 iterations 112.89178314208985
Epoch 11 / 100: avg. loss of last 5 iterations 112.31478576660156
Epoch 12 / 100: avg. loss of last 5 iterations 111.74916534423828
Epoch 13 / 100: avg. loss of last 5 iterations 111.19446868896485
Epoch 14 / 100: avg. loss of last 5 iterations 110.63250427246093
Epoch 15 / 100: avg. loss

In [ ]:
model.eval()
predicted = model(x)
predicted = predicted.data.numpy()
plt.plot(x,predicted)
plt.show()